In [81]:
def check_date_format(date):
    """
    Ensures that a date is correctly formatted and that an end date is later than a start date.
    """
    for item in items:
        if len(item) > 10:
            format = '%Y-%m-%dT%H:%M:%S'
        else:
            format = '%Y-%m-%d'
        try:
            if item != datetime.strptime(item, format).strftime(format):
                raise ValueError
            return True
        except ValueError:
            print('The date value ' + item + ' is in an incorrect format. Use YYYY-MM-DD or YYYY-MM-DDTHH:MM:SS.')
    items = date.replace(' ', '').split(',')
    if len(items) == 2:
        try:
            assert items[1] > items[0]
        except:
            print('Your end date ' + items[1] + ' must be after your start date ' + items[0] + '.')


    
def process_dates(datefield):
    """
    Takes a line-separated list of dates from a text area. Ranges must have start and end
    dates separated with commas.
    
    Output is a list of dates. Dates must be in date or datetime format. Mixtures of normal
    and precise dates are placed in separate normal and precise objects. Otherwise, all dates
    are assumed to be normal.
    """ 
    import os, json, requests
    from datetime import datetime

    date = datefield.strip().split('\n')
    new_date = []
    d = {}

    # Validate all the date formats individually
    for item in date:
        check_date_format(item)

    # Determine if the datelist is a mix of normal and precise dates
    contains_precise = []
    for item in date:
        if ',' in item:
            start, end = item.replace(' ', '').split(',')
            if len(start) > 10 or len(end) > 10:
                contains_precise.append('precise')
            else:
                contains_precise.append('normal')
        elif len(item) > 10:
            contains_precise.append('precise')
        else:
            contains_precise.append('normal')

    # Handle a mix of normal and precise dates
    if 'normal' in contains_precise and 'precise' in contains_precise:
        d['normal'] = []
        d['precise'] = []
        for item in date:
            if ',' in item:
                start, end = item.replace(' ', '').split(',')
                if len(start) > 10 or len(end) > 10:
                    d['precise'].append({'start': start, 'end': end})
                else:
                    d['normal'].append({'start': start, 'end': end})
            else:
                if len(item) > 10:
                    d['precise'].append(item)
                else:
                    d['normal'].append(item)
        new_date.append(d)
    # Handle only precise dates
    elif 'precise' in contains_precise:
        d['precise'] = []
        for item in date:
            if ',' in item:
                start, end = item.replace(' ', '').split(',')
                d['precise'].append({'start': start, 'end': end})
            else:
                d['precise'].append(item)
        new_date.append(d)
    # Handle only normal dates
    else:
        for item in date:
            if ',' in item:
                start, end = item.replace(' ', '').split(',')
                new_date.append({'start': start, 'end': end})
            else:
                new_date.append(item)

    return new_date

datefield = """
2013-01-01T12:00:30
2013-01-01T12:00:31
"""
date = process_dates(datefield)
print(date)

[{'precise': ['2013-01-01T12:00:30', '2013-01-01T12:00:31']}]


In [61]:
import os, json, requests
from jsonschema import validate, FormatChecker
schema_file = 'https://github.com/whatevery1says/manifest/raw/master/schema/global/global.json'
schema = json.loads(requests.get(schema_file).text)
# Remove all the non-date properties from the global schema
remove = ['altTitle', 'group', '_id', 'label', 'title', 'description', 'namespace', 'note', 'notes', 'path', 'refLocation']
for item in remove:
    del schema[item]

schema = {
    "definitions": {
        "daterange": {
            "type": "object",
            "properties": {
                "start": {
                "type": "string",
                "format": "date"
                },
                "end": {
                "type": "string",
                "format": "date"
                }
            },
            "required": [
                "start",
                "end"
            ]
        },
        "normal": {
            "type": "object",
            "properties": {
                "normal": {
                    "type": "array",
                    "items": {
                        "oneOf": [
                        {
                            "type": "string",
                            "format": "date"
                        },
                        {
                            "$ref": "#/definitions/daterange"
                        }]
                    }
                }
            }
        }
    },
    "type": "object",
    "properties": {
        "date": {
          "type": "array",
          "items": {
            "oneOf": [{
              "type": "string",
              "format": "date"
            },
            {
                "type": "string",
                "format": "date-time"
            },{
                "$ref": "#/definitions/daterange"
            }]
        }
      }
    }
}
f = {
    "date": [
        {"normal": ["2013-01-01"]}
    ]
}
try:
    validate(f, schema, format_checker=FormatChecker())
    print('Valid')
except:
    print('Not Valid')

Not Valid


In [96]:
from datetime import datetime
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Checkbox, Box, Dropdown, Textarea, fixed, interact, interactive, interact_manual, Label, Layout, Textarea

# Override ipywidgets styles
display(HTML('''
<style>
textarea {min-height: 110px !important;}
/* Allow extra-long labels */
.widget-label {min-width: 20ex !important; font-weight: bold;}
.widget-checkbox {padding-right: 300px !important;}
/* Classes for toggling widget visibility */
.hidden {display: none;}
.visible {display: flex;}
</style>
'''))


class ConfigForm(object):
    # Define widgets
    caption = widgets.HTML(value='<h4>Global Congfiguration</h4>')
    button = widgets.Button(description='Submit')
    title = widgets.Text(value='')
    altTitle = widgets.Text(value='')
    label = widgets.Text(value='')
    description = widgets.Textarea(value='')
    precise = widgets.Checkbox(value=False)
    date = widgets.Textarea(placeholder='Place each date on a separate line. Date ranges should use the separate start and end dates with commas. Valid formats are YYYY-MM-DD and YYYY-MM-DDTHH:MM:SS."')
    notes = widgets.Textarea(placeholder='Place each note on a separate line.')
    
    # Configure widget layout
    flex = Layout(display='flex', flex_flow='row', justify_content='space-between')

    # Assemble widgets in Boxes
    global_form_items = [
        Box([Label(value='Title:'), title], layout=flex),
        Box([Label(value='altTitle:'), altTitle], layout=flex),
        Box([Label(value='Description:'), description], layout=flex),
        Box([Label(value='Label:'), label], layout=flex),
        Box([Label(value='Date:'), date], layout=flex),
        Box([Label(value='Notes:'), notes], layout=flex)
    ]
    
    # Initialise the class object
    def __init__(self, object):
        self.caption = ConfigForm.caption
        self.button = ConfigForm.button
        self.title = ConfigForm.global_form_items[0]
        self.altTitle = ConfigForm.global_form_items[1]
        self.description = ConfigForm.global_form_items[2]
        self.label = ConfigForm.global_form_items[3]
        self.date = ConfigForm.global_form_items[4]
        self.notes = ConfigForm.global_form_items[5]               

                
        # Ensure that a date is correctly formatted and start dates precede end dates
        def check_date_format(date):
            items = date.replace(' ', '').split(',')
            for item in items:
                if len(item) > 10:
                    format = '%Y-%m-%dT%H:%M:%S'
                else:
                    format = '%Y-%m-%d'
                try:
                    if item != datetime.strptime(item, format).strftime(format):
                        raise ValueError
                    return True
                except ValueError:
                    print('The date value ' + item + ' is in an incorrect format. Use YYYY-MM-DD or YYYY-MM-DDTHH:MM:SS.')
            if len(items) == 2:
                try:
                    assert items[1] > items[0]
                except:
                    print('Your end date ' + items[1] + ' must be after your start date ' + items[0] + '.')


        # Transform textarea with dates to a valid schema array
        def process_dates(datefield):
            date = datefield.strip().split('\n')
            new_date = []
            d = {}

            # Validate all the date formats individually
            for item in date:
                check_date_format(item)

            # Determine if the datelist is a mix of normal and precise dates
            contains_precise = []
            for item in date:
                if ',' in item:
                    start, end = item.replace(' ', '').split(',')
                    if len(start) > 10 or len(end) > 10:
                        contains_precise.append('precise')
                    else:
                        contains_precise.append('normal')
                elif len(item) > 10:
                    contains_precise.append('precise')
                else:
                    contains_precise.append('normal')
            # Handle a mix of normal and precise dates
            if 'normal' in contains_precise and 'precise' in contains_precise:
                d['normal'] = []
                d['precise'] = []
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        if len(start) > 10 or len(end) > 10:
                            d['precise'].append({'start': start, 'end': end})
                        else:
                            d['normal'].append({'start': start, 'end': end})
                    else:
                        if len(item) > 10:
                            d['precise'].append(item)
                        else:
                            d['normal'].append(item)
                new_date.append(d)
            # Handle only precise dates
            elif 'precise' in contains_precise:
                d['precise'] = []
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        d['precise'].append({'start': start, 'end': end})
                    else:
                        d['precise'].append(item)
                new_date.append(d)
            # Handle only normal dates
            else:
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        new_date.append({'start': start, 'end': end})
                    else:
                        new_date.append(item)

            return new_date
            
        
        def split_list(str):
            seq = str.split('\n')
            for key, value in enumerate(seq):
                seq[key] = value.strip()
            return seq

        self.title = ConfigForm.global_form_items[0]
        self.altTitle = ConfigForm.global_form_items[1]
        self.description = ConfigForm.global_form_items[2]
        self.label = ConfigForm.global_form_items[3]
        self.date = ConfigForm.global_form_items[4]
        self.notes = ConfigForm.global_form_items[5]

        
        def handle_submit(values):
            # Save the form values in a dict
            self.values = {'action': 'global'}
            self.values['title'] = ConfigForm.title.value.strip()
            self.values['altTitle'] = ConfigForm.altTitle.value.strip()
            self.values['description'] = ConfigForm.description.value.strip()
            self.values['label'] = ConfigForm.label.value.strip()
            self.values['date'] = process_dates(ConfigForm.date.value.strip())
            self.values['notes'] = split_list(ConfigForm.notes.value.strip())
    
        # Initialise widgets in the container Box
        self.form = Box([self.title, self.altTitle, self.description, self.label, self.date, self.notes],
                   layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 2px',
                    align_items='stretch',
                    width='50%'))

        # Modify the CSS and set up some helper variables
        box = self.form
        # box.children[3].add_class('hidden')
        action_field = box.children[0].children[1]
        
        # Display the form and watch for changes
        display(self.caption)
        display(box)
        display(self.button)
        self.button.on_click(handle_submit)

# Instantiate the form - values accessible with e.g. config.values['delete_id]
config = ConfigForm(object)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [97]:
print(config.values['date'])

['2013-01-01', '2013-01-02', {'start': '2013-01-04', 'end': '2013-01-31'}]
